Programme utilisant le modèle CatBoostRegressor

Librairies

In [2]:
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, TimeSeriesSplit

'ok'

'ok'

Cross Validation du modèle

In [3]:
#On charge les données

dataframe = pd.read_csv('Data\BDDLAST.csv', engine='python', )
dataset = dataframe.values

#On donne ce que l'on doit trouver (y) et avec quoi (X)
mois6 = dataframe['mois_6']
sem6 = dataframe['numeroJourDansSemaine_6']
mois12 = dataframe['mois_12']
mois5 = dataframe['mois_5']
noel = dataframe['event_J-2 NOEL']
sem2 = dataframe['numeroJourDansSemaine_2']
an = dataframe['event_J+1 Pont Nouvel An']
mois4 = dataframe['mois_4']
paques = dataframe['holiday_Lundi de paques']
mois9 = dataframe['mois_9']
sem5 = dataframe['numeroJourDansSemaine_5']

df = pd.concat([mois6,sem6,mois12,mois5,noel,sem2,an,mois4,paques,mois9,sem5],axis=1)
X=df

#X = dataframe.drop('quantiteVendue', axis=1)
y = dataframe['quantiteVendue']

#On définit la MAPE qui sert quantifier l'erreur: pourcentage moyen de différence

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)   
    a = np.array(y_true)
    b = np.array(y_pred)
    mean = (a + b)/2.0
    return 100*np.mean(np.abs((y_true - y_pred) / np.maximum(mean, 1)))

#On définit la RMSE qui sert quantifier l'erreur: valeur moyenne de différence

def root_mean_square_error(y_pred, y_true):
    return math.sqrt(mean_squared_error(y_pred, y_true))

#Nous crééons les différents splits

# X_groups = pd.qcut(X.index, 100, labels=False)
pred = pd.Series([np.nan] * len(X), index=X.index)

n_predictions = 4
periode_prediction = 15

tss = TimeSeriesSplit(n_splits=n_predictions)
m_err=0
for train_index, test_index in tss.split(X): #, groups=X_groups):
    test_index = test_index[0:periode_prediction]
    
    X_train = X.loc[train_index]
    y_train = y.loc[train_index]
    X_test = X.loc[test_index]
    y_test = y.loc[test_index]
    
    # Testons à chaque fois le modèle
    
    model=CatBoostRegressor(iterations=1000, depth=2, loss_function="RMSE", verbose=False, random_seed=2, od_type='Iter', od_wait=50, boosting_type='Plain')
    model.fit(X_train, y_train)

    # Puis nous ajoutons à un axe vierge tous les résultats
    
    pred.loc[test_index] = model.predict(X_test)

#Enfin nous nous comparons les résultats
    
predicted_index = ~pred.isnull()
relative_error = mean_absolute_error(y[predicted_index], pred[predicted_index]) / np.mean(y)
mape=mean_absolute_percentage_error(y[predicted_index], pred[predicted_index])
print('Relative error for model {}: {:.2f}% '.format(model.__class__.__name__, relative_error*100))
print('Relative MAPE error for model: {:.2f}% '.format(mape))

KeyError: 'quantiteVendue'

Split "taille" pour tester le modèle

In [132]:
#80 de valeur train et 20 de test

taille=len(X)*0.80

#Création de toutes les variables

X_train = X[dataframe.index < taille]
y_train = y[dataframe.index < taille]

X_test = X[dataframe.index >= taille]
y_test = y[dataframe.index >= taille]

#Gestion de tous les paramètres du modèle
model = CatBoostRegressor(iterations=1000,
                        learning_rate=None,
                        depth=2,
                        l2_leaf_reg=None,
                        model_size_reg=None,
                        rsm=None,
                        loss_function='RMSE',
                        border_count=None,
                        feature_border_type=None,
                        fold_permutation_block_size=None,
                        od_pval=None,
                        od_wait=50,
                        od_type='Iter',
                        nan_mode=None,
                        counter_calc_method=None,
                        leaf_estimation_iterations=None,
                        leaf_estimation_method=None,
                        thread_count=None,
                        random_seed=2,
                        use_best_model=None,
                        verbose=False,
                        logging_level=None,
                        metric_period=None,
                        ctr_leaf_count_limit=None,
                        store_all_simple_ctr=None,
                        max_ctr_complexity=None,
                        has_time=None,
                        allow_const_label=None, 
                        one_hot_max_size=None,
                        random_strength=None,
                        name=None,
                        ignored_features=None,
                        train_dir=None,
                        custom_metric=None,
                        eval_metric='MAPE',
                        bagging_temperature=None,
                        save_snapshot=None,
                        snapshot_file=None,
                        fold_len_multiplier=None,
                        used_ram_limit=None,
                        gpu_ram_part=None,
                        allow_writing_files=None,
                        final_ctr_computation_mode=None,
                        approx_on_full_history=None,
                        boosting_type='Plain',
                        simple_ctr=None,
                        combinations_ctr=None,
                        per_feature_ctr=None,
                        task_type=None,
                        device_config=None,
                        devices=None,
                        bootstrap_type='Bernoulli',
                        subsample=None,
                        max_depth=None,
                        n_estimators=None,
                        num_boost_round=None,
                        num_trees=None,
                        colsample_bylevel=None,
                        random_state=None,
                        reg_lambda=None,
                        objective=None,
                        eta=None,
                        max_bin=None,
                        gpu_cat_features_storage=None,
                        data_partition=None,
                        metadata=None)

#Entrainement du modèle

model.fit(X_train, y_train, use_best_model=True)

#Comparatif réel/prédit

predictions = model.predict(X_test)
error = None
mape = mean_absolute_percentage_error(y_test, predictions)
rmse = root_mean_square_error(y_test, predictions)
print('Relative MAPE error for model: {:.2f}% '.format(mape))
print('Relative RMSE error for model: {:.2f} '.format(rmse))

You should provide test set for use best model. use_best_model parameter swiched to false value.
Relative MAPE error for model: 14.32% 
Relative RMSE error for model: 86.10 
